In [1]:
import pandas as pd
import json
import numpy as np
import jenks
import copy

In [2]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('Kod').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('Kod').to_dict(orient='index')
okr = pd.read_csv('../scratch/okres_meta.csv').set_index('Kod').to_dict(orient='index')

In [18]:
okr

{3100: {'gml_id': 'OK.3100',
  'Nazev': 'Hlavní město Praha',
  'Nespravny': nan,
  'KrajKod': 31,
  'VuscKod': 19,
  'PlatiOd': '2017-12-06T00:00:00',
  'PlatiDo': nan,
  'IdTransakce': 2181367,
  'GlobalniIdNavrhuZmeny': 1602521,
  'NutsLau': 'CZ0100',
  'DatumVzniku': '1960-04-11T00:00:00'},
 3201: {'gml_id': 'OK.3201',
  'Nazev': 'Benešov',
  'Nespravny': nan,
  'KrajKod': 32,
  'VuscKod': 27,
  'PlatiOd': '2017-11-23T00:00:00',
  'PlatiDo': nan,
  'IdTransakce': 2152260,
  'GlobalniIdNavrhuZmeny': 1580290,
  'NutsLau': 'CZ0201',
  'DatumVzniku': '1960-04-11T00:00:00'},
 3202: {'gml_id': 'OK.3202',
  'Nazev': 'Beroun',
  'Nespravny': nan,
  'KrajKod': 32,
  'VuscKod': 27,
  'PlatiOd': '2018-06-04T00:00:00',
  'PlatiDo': nan,
  'IdTransakce': 2437657,
  'GlobalniIdNavrhuZmeny': 1767814,
  'NutsLau': 'CZ0202',
  'DatumVzniku': '1960-04-11T00:00:00'},
 3203: {'gml_id': 'OK.3203',
  'Nazev': 'Kladno',
  'Nespravny': nan,
  'KrajKod': 32,
  'VuscKod': 27,
  'PlatiOd': '2017-12-04T00:00:

In [3]:
with open('../scratch/okrsky.geojson', encoding='utf-8') as f:
    jsn = json.load(f)

In [4]:
kill = ['fid', 'gml_id', 'Kod', 'Nespravny', 'ObecKod', 'MomcKod', 'Poznamka', 'PlatiOd', 'PlatiDo', 'IdTransakce', 'GlobalniIdNavrhuZmeny']
mcmo_dump = []
for f in jsn['features']:
    f['properties']['Obec'] = ob[f['properties']['ObecKod']]['Nazev']
    f['properties']['Okres'] = okr[ob[f['properties']['ObecKod']]['OkresKod']]['Nazev']
    f['properties']['Momc'] = None
    f['properties']['isMomc'] = False
    f['properties']['okid'] = str(f['properties']['ObecKod']) + '_' + str(f['properties']['Cislo'])
    
    if f['properties']['MomcKod'] is not None:
        f['properties']['Momc'] = momc[f['properties']['MomcKod']]['Nazev']
        n = copy.deepcopy(f)
        n['properties']['isMomc'] = True
        n['properties']['okid'] = str(f['properties']['MomcKod']) + '_' + str(f['properties']['Cislo'])
        
        for k in kill:
            if k in n['properties']:
                del n['properties'][k]
        
        mcmo_dump.append(n)

    for k in kill:
        if k in f['properties']:
            del f['properties'][k]
            
jsn['features'].extend(mcmo_dump)

In [5]:
okrs = pd.read_csv('../scratch/okrsky_cisla.csv')

In [6]:
okrs['okid'] = okrs.KODZASTUP.astype('str') + '_' + okrs.CIS_OKRSEK.astype('str')

In [7]:
cls = ['CIS_OBEC', 'CIS_OBVODU', 'CIS_OKRSEK', 'DATUM_CAS_ZPRAC', 'OPAKOVANE', 'PORADI_ZPRAC']
okrs.drop(cls, inplace=True, axis=1)

In [8]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [9]:
for fe in jsn['features']:
    if fe['properties']['okid'] in okrs:
        fe['properties'].update(okrs[fe['properties']['okid']])
    else:
        print(fe['properties']['okid'])

565181_1
577596_1
577596_2
549886_1
581950_1
566845_1
530662_1
592935_1
571318_1
578959_1
566837_1
549169_1
531774_1
545422_1
571075_1
532258_1
536474_1
555177_1
557722_1
557722_2
581526_1
593044_1
503941_1
555321_2
555321_3
555321_4
555321_5
555321_6
555321_7
555321_8
555321_9
555321_10
555321_11
555321_12
555321_13
555321_14
555321_15
555321_16
555321_17
555321_18
555321_19
555321_20
555321_21
555321_22
555321_23
555321_24
555321_25
555321_26
555321_27
555321_28
555321_29
555321_30
555321_31
555321_32
555321_33
555321_34
555321_35
555321_36
555321_37
555321_38
555321_39
555321_40
555321_41
555321_42
555321_43
555321_44
555321_45
555321_46
555321_1
555321_47
555321_48
555321_49
555321_50
556904_1
556904_2
556904_3
556904_4
556904_5
556904_6
556904_7
556904_8
556904_9
556904_10
556904_11
556904_12
556904_13
556904_14
556904_15
556904_16
556904_17
556904_18
556904_19
556904_20
556904_21
556904_22
556904_23
556904_24
556904_25
556904_26
556904_27
556904_28
556904_29
556904_30
556904_31
5

In [10]:
jsn['features'] = list(filter(lambda x: 'ZAPSANI_VOLICI' in x['properties'], jsn['features']))

In [11]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))

In [12]:
#breaks
k = 5
nbAttempt = 4

target = open('../js/breaks.js', 'w', encoding='utf-8')
target.truncate()
target.write('var breaks = {\n')

partaje = ['KDU-ČSL','KSČM','ČSSD','ODS','ANO','STAN','SPD','TOP 09','Piráti', 'Ostatní']

for i in partaje:
    print('partaj ' + str(i))
    vals = []
    for ftr in jsn['features']:
        try:
            vals.append(ftr['properties']['HL_' + i] / ftr['properties']['PLATNE_HLASY'])
        except:
            print('omit')
        
    vals = np.trim_zeros(vals)
    if np.mean(vals) == 0:
        continue
    else:
        brks = [0, 0, 0, 0]
        try:
            casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
            brks = casp.breaks
        except:
            pass
        target.write('"HL_' + i + '": ' + str(brks) + ',\n')
        
vals = []
print('ucast')
for ftr in jsn['features']:
    try:
        vals.append(ftr['properties']['PLATNE_HLASY'] / ftr['properties']['ZAPSANI_VOLICI'])
    except:
        print('omit')

vals = np.trim_zeros(vals)
if np.mean(vals) == 0:
    pass
else:
    brks = [0, 0, 0, 0]
    try:
        casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
        brks = casp.breaks
    except:
        pass
    target.write('"UCAST": ' + str(brks) + ',\n')       
target.write("};")
target.close()

partaj KDU-ČSL
Running Jenks-Caspall natural breaks...
**Attempt number 1
Step 1 : kmeans++ initalization, GVF = 0.921619
Step 2 : kmeans complete in 18 iterations and 1378 moves, GVF = 0.964257
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.964257
Running Jenks-Caspall natural breaks...
**Attempt number 2
Step 1 : kmeans++ initalization, GVF = 0.926825
Step 2 : kmeans complete in 21 iterations and 1682 moves, GVF = 0.964271
Step 3 : Forcing completed in 2 cycles and 3 moves, GVF = 0.964271
Running Jenks-Caspall natural breaks...
**Attempt number 3
Step 1 : kmeans++ initalization, GVF = 0.947727
Step 2 : kmeans complete in 21 iterations and 1409 moves, GVF = 0.964271
Step 3 : Forcing completed in 2 cycles and 3 moves, GVF = 0.964271
Running Jenks-Caspall natural breaks...
**Attempt number 4
Step 1 : kmeans++ initalization, GVF = 0.936545
Step 2 : kmeans complete in 22 iterations and 1780 moves, GVF = 0.964271
Step 3 : Forcing completed in 2 cycles and 3 moves, GVF = 0.9642